In [3]:
%%capture

!pip install minisom
!pip install spotipy

In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from minisom import MiniSom
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import seaborn as sns
import pickle
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.models import Sequential, model_from_json
from tensorflow.keras.layers import Input, Dense, Activation
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy
import json

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [37]:
#data loaded from repo
songs = pd.read_csv('https://raw.githubusercontent.com/spotify-song-suggester1/machine_learning/master/data/SpotifyFeatures.csv?token=APCSMRNWVE65U7D6AUVOLY27D4DIO')

In [25]:
songs.head()

,Unnamed: 0,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,0,Movie,Henri Salvador,C'est beau de faire un Show,0BRjO6ga9RKCKjfDqeFgWV,0,0.611,0.389,99373,0.910,0.000,C#,0.3460,-1.828,Major,0.0525,166.969,4/4,0.814
1,1,Movie,Martin & les fées,Perdu d'avance (par Gad Elmaleh),0BjC1NfoEOOusryehmNudP,1,0.246,0.590,137373,0.737,0.000,F#,0.1510,-5.559,Minor,0.0868,174.003,4/4,0.816
2,2,Movie,Joseph Williams,Don't Let Me Be Lonely Tonight,0CoSDzoNIKCRs124s9uTVy,3,0.952,0.663,170267,0.131,0.000,C,0.1030,-13.879,Minor,0.0362,99.488,5/4,0.368
3,3,Movie,Henri Salvador,Dis-moi Monsieur Gordon Cooper,0Gc6TVm52BwZD07Ki6tIvf,0,0.703,0.240,152427,0.326,0.000,C#,0.0985,-12.178,Major,0.0395,171.758,4/4,0.227
4,4,Movie,Fabien Nataf,Ouverture,0IuslXpMROHdEPvSl1fTQK,4,0.950,0.331,82625,0.225,0.123,F,0.2020,-21.150,Major,0.0456,140.576,4/4,0.390


In [13]:
songs['popularity'].value_counts()

0      6312
50     5415
53     5414
51     5401
52     5342
       ... 
96        8
94        7
99        4
98        3
100       2
Name: popularity, Length: 101, dtype: int64

In [16]:
songs.shape

(232725, 19)

In [38]:
#Loading and preparing the data

#cleaning songs df

songs = songs.drop(labels=['track_id', 'track_name', 'time_signature', 'artist_name', 'key'], axis=1)



NameError: ignored

In [39]:
songs.head()

,Unnamed: 0,genre,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,mode,speechiness,tempo,valence
0,0,Movie,0,0.611,0.389,99373,0.910,0.000,0.3460,-1.828,Major,0.0525,166.969,0.814
1,1,Movie,1,0.246,0.590,137373,0.737,0.000,0.1510,-5.559,Minor,0.0868,174.003,0.816
2,2,Movie,3,0.952,0.663,170267,0.131,0.000,0.1030,-13.879,Minor,0.0362,99.488,0.368
3,3,Movie,0,0.703,0.240,152427,0.326,0.000,0.0985,-12.178,Major,0.0395,171.758,0.227
4,4,Movie,4,0.950,0.331,82625,0.225,0.123,0.2020,-21.150,Major,0.0456,140.576,0.390


In [45]:
#creating a scaled features df from the song df

features = songs.drop(labels=['popularity', 'genre', 'mode'], axis=1)
scaler = MinMaxScaler()
features_scaled = scaler.fit_transform(features)

# implementing kmeans
np.random.seed(42)
kmeans = KMeans(n_clusters=7)
kmeans.fit(features_scaled)

#cluster predictions for each song
y_kmeans = kmeans.predict(features_scaled)

#adding song kmean group label to song df 
songs['label'] = y_kmeans
songs = songs.sample(frac=1)

#train test split
X_train, X_test, y_train, y_test = train_test_split(features_scaled, y_kmeans, test_size=0.2)

y_train_binary = to_categorical(y_train)
y_test_binary = to_categorical(y_test)

# #neural network
model = Sequential([
    Input(shape=(X_train.shape[1])),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(7, activation='softmax')
])

inputs = Input(shape=(10,))

model.compile('nadam', 'categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train_binary, epochs=10, batch_size=16, validation_split=.2)

Epoch 1/10
9309/9309 [==============================] - 18s 2ms/step - loss: 0.1028 - accuracy: 0.9607 - val_loss: 0.0497 - val_accuracy: 0.9823
Epoch 2/10
9309/9309 [==============================] - 17s 2ms/step - loss: 0.0564 - accuracy: 0.9760 - val_loss: 0.0401 - val_accuracy: 0.9827
Epoch 3/10
9309/9309 [==============================] - 18s 2ms/step - loss: 0.0496 - accuracy: 0.9792 - val_loss: 0.0609 - val_accuracy: 0.9741
Epoch 4/10
9309/9309 [==============================] - 18s 2ms/step - loss: 0.0472 - accuracy: 0.9803 - val_loss: 0.0434 - val_accuracy: 0.9828
Epoch 5/10
9309/9309 [==============================] - 18s 2ms/step - loss: 0.0434 - accuracy: 0.9816 - val_loss: 0.0332 - val_accuracy: 0.9860
Epoch 6/10
9309/9309 [==============================] - 18s 2ms/step - loss: 0.0411 - accuracy: 0.9826 - val_loss: 0.0374 - val_accuracy: 0.9850
Epoch 7/10
9309/9309 [==============================] - 17s 2ms/step - loss: 0.0389 - accuracy: 0.9838 - val_loss: 0.0457 - val_ac

In [46]:
songs

,Unnamed: 0,genre,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,mode,speechiness,tempo,valence,label
10186,10186,Alternative,49,0.41300,0.612,217120,0.6590,0.402000,0.1100,-9.714,Minor,0.0476,144.904,0.2540,5
106805,106805,Opera,10,0.95600,0.145,416173,0.0463,0.812000,0.1030,-24.106,Minor,0.0471,97.374,0.0397,2
101600,101600,Children’s Music,44,0.02910,0.554,299890,0.5230,0.000029,0.1310,-7.431,Minor,0.0368,89.925,0.1040,4
161485,161485,Reggaeton,35,0.20100,0.811,239427,0.8070,0.007680,0.1990,-5.766,Minor,0.0436,120.009,0.8880,6
68357,68357,Hip-Hop,62,0.25500,0.441,434960,0.4030,0.000013,0.1300,-11.180,Minor,0.1470,139.920,0.1580,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126324,126324,Classical,36,0.97700,0.079,312200,0.0406,0.911000,0.1090,-24.441,Minor,0.0441,174.490,0.0351,2
112727,112727,Pop,65,0.00448,0.447,162720,0.9670,0.000000,0.4870,-2.706,Major,0.0793,179.991,0.6060,5
87498,87498,Rap,77,0.51100,0.732,221933,0.7240,0.000000,0.0491,-2.104,Major,0.1010,140.130,0.8080,6
168266,168266,Comedy,32,0.81300,0.457,398715,0.9790,0.000000,0.7410,-6.785,Major,0.9350,91.190,0.1340,3


In [23]:
songs

In [47]:
songs['label'] = y_kmeans

songs = songs.sample(frac=1)
songs['label'].value_counts()

5    60104
6    46581
1    41384
4    37024
2    23402
0    14298
3     9932
Name: label, dtype: int64

In [48]:
model.evaluate(X_test, y_test_binary)

1455/1455 [==============================] - 2s 1ms/step - loss: 0.0510 - accuracy: 0.9777


[0.05100386589765549, 0.9776560068130493]

In [49]:
# Predicting from API
# credentials

client_credentials_manager = SpotifyClientCredentials(client_id = '18bf732ea7ab485e91a45dfe7b75d5ec', client_secret = '61d0f5271ab34e7aa519f1bebd81d3c9')
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [50]:
# pulling a track from a playlist

playlist = sp.playlist('open.spotify.com/playlist/2YRe7HRKNRvXdJBp9nXFza')

track_info = dict(song=playlist['tracks']['items'][12]['track']['name'], 
                  artist=playlist['tracks']['items'][12]['track']['album']['artists'][0]['name'],
                  uri=playlist['tracks']['items'][12]['track']['uri'])

track_info

{'artist': 'Billie Eilish',
 'song': 'bad guy',
 'uri': 'spotify:track:2Fxmhks0bxGSBdJ92vM42m'}

In [51]:
test_track_uri = sp.track('spotify:track:6v3KW9xbzN5yKLt9YKDYA2')
test_track_features = sp.audio_features('spotify:track:6v3KW9xbzN5yKLt9YKDYA2')
test_track_features

[{'acousticness': 0.0392,
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/6v3KW9xbzN5yKLt9YKDYA2',
  'danceability': 0.759,
  'duration_ms': 190800,
  'energy': 0.548,
  'id': '6v3KW9xbzN5yKLt9YKDYA2',
  'instrumentalness': 0,
  'key': 9,
  'liveness': 0.0828,
  'loudness': -6.049,
  'mode': 0,
  'speechiness': 0.029,
  'tempo': 116.967,
  'time_signature': 4,
  'track_href': 'https://api.spotify.com/v1/tracks/6v3KW9xbzN5yKLt9YKDYA2',
  'type': 'audio_features',
  'uri': 'spotify:track:6v3KW9xbzN5yKLt9YKDYA2',
  'valence': 0.749}]

In [52]:
# pulling audio features for selected track

track_features = sp.audio_features(track_info['uri'])

track_features

[{'acousticness': 0.328,
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/2Fxmhks0bxGSBdJ92vM42m',
  'danceability': 0.701,
  'duration_ms': 194088,
  'energy': 0.425,
  'id': '2Fxmhks0bxGSBdJ92vM42m',
  'instrumentalness': 0.13,
  'key': 7,
  'liveness': 0.1,
  'loudness': -10.965,
  'mode': 1,
  'speechiness': 0.375,
  'tempo': 135.128,
  'time_signature': 4,
  'track_href': 'https://api.spotify.com/v1/tracks/2Fxmhks0bxGSBdJ92vM42m',
  'type': 'audio_features',
  'uri': 'spotify:track:2Fxmhks0bxGSBdJ92vM42m',
  'valence': 0.562}]

In [53]:
# trimming to only relevant features, need to drop unnamed

track = [track_features[0][i] for i in features]

track

KeyError: ignored

[0.759, 0.548, -6.049, 0.029, 0.0392, 0, 0.0828, 0.749, 116.967, 4]